In [1]:
from model import Model
model = Model() # TODO: Model endpoint

In [2]:
# # Creating the dataset
# import json
# import random

# data = []

# for index in range(1, 101):
#     a = random.randint(10**(index-1), 10**index - 1)
#     b = random.randint(10**(index-1), 10**index - 1)
#     question = f"{a} + {b} = "
#     answer = a + b
#     data.append({
#         "index": index,
#         "question": question,
#         "answer": str(answer)
#     })

# with open("100_digit_addition_testset.json", "w") as file:
#     json.dump(data, file, indent=4)

# print("JSON file created successfully.")


In [2]:
import json
with open("100_digit_addition_testset.json", "r") as file:
    data = json.load(file)

In [3]:
from tqdm import tqdm

answers = []
for i in tqdm(range(100)):
    answers.append(model.get_completions(data[i]['question'] + "\nThink step by step."))

  0%|          | 0/100 [00:00<?, ?it/s]

EXCEPTION 0 in GET GPTV COMPLETION:  'choices'
Completion {'error': {'message': 'max_tokens is too large: 8096. This model supports at most 4096 completion tokens, whereas you provided 8096.', 'type': None, 'param': 'max_tokens', 'code': None}}


In [ ]:
model.get_completions(data[0]['question'] + "\nThink step by step.")

"Sure, let's break it down step by step:\n\n1. Start with the number 4.\n2. Add 1 to 4, which gives you 5.\n3. Add another 1 to 5, which gives you 6.\n4. Add another 1 to 6, which gives you 7.\n5. Add another 1 to 7, which gives you 8.\n6. Finally, add another 1 to 8, which gives you 9.\n\nSo, 4 + 5 = 9."

In [ ]:
import re
def extract_final_answer(full_answer):
    # Use regex to extract the last number
    match = re.findall(r'\d+', full_answer)

    if match:
        last_number = match[-1]  # Get the last matched number
        return last_number
    else:
        print("No match found.")
    return -1

In [ ]:
res = model.get_completions(data[0]['question'] + "\nThink step by step.")
print("Res\n", res)
print("\nextract_final_answer(res)\n", extract_final_answer(res))


Res
 Sure, let's break it down step by step:

1. Start with the number 4.
2. Add 1 to 4, which gives you 5.
3. Add another 1 to 5, which gives you 6.
4. Add another 1 to 6, which gives you 7.
5. Add another 1 to 7, which gives you 8.
6. Finally, add another 1 to 8, which gives you 9.

So, 4 + 5 = 9.

extract_final_answer(res)
 9


In [ ]:
import json

def calculate_accuracy(correct_answers, predictions):
    total_questions = len(correct_answers)
    correct_predictions = sum(1 for correct, pred in zip(correct_answers, predictions) if correct == pred)
    accuracy = correct_predictions / total_questions
    return accuracy

def calculate_accuracy_per_section(correct_answers, predictions, section_size=10):
    accuracies = []
    total_sections = len(correct_answers) // section_size
    for section in range(total_sections):
        start_index = section * section_size
        end_index = start_index + section_size
        section_correct_answers = correct_answers[start_index:end_index]
        section_predictions = predictions[start_index:end_index]
        section_accuracy = calculate_accuracy(section_correct_answers, section_predictions)
        accuracies.append(section_accuracy)
    return accuracies

# Load the JSON file with questions and correct answers
with open("100_digit_addition_testset.json", "r") as file:
    data = json.load(file)

# Extract correct answers
correct_answers = [item["answer"] for item in data]

predictions = [extract_final_answer(item) for item in answers]

# Calculate overall accuracy
overall_accuracy = calculate_accuracy(correct_answers, predictions)

# Calculate accuracy per section of tens
section_accuracies = calculate_accuracy_per_section(correct_answers, predictions)

# Print and save the accuracies
print(f"Overall Accuracy: {overall_accuracy}")
print(f"Section Accuracies: {section_accuracies}")

# Save accuracies to a file
accuracies_data = {
    "overall_accuracy": overall_accuracy,
    "section_accuracies": section_accuracies,
    "predictions": predictions,
    "answers": answers
}

with open("accuracies_cot_100000.json", "w") as file:
    json.dump(accuracies_data, file, indent=4)

print("Accuracies saved successfully.")

Overall Accuracy: 0.39
Section Accuracies: [0.9, 0.9, 0.4, 0.6, 0.2, 0.2, 0.1, 0.1, 0.1, 0.4]
Accuracies saved successfully.


In [ ]:
print(answers[58])

To solve the addition of two large numbers step by step, we can align them by their digits and add them column by column from right to left, carrying over any values as needed.

Let's write the numbers one below the other:

```
  88356247731782694958049429079553866383573654648829635163142
+ 47781035507177048288144828253014211541526542770035318534047
-------------------------------------------------------------
```

Now, let's add them column by column from right to left:

1. **Rightmost column:**
   - 2 + 7 = 9

2. **Next column:**
   - 4 + 4 = 8

3. **Next column:**
   - 1 + 0 = 1

4. **Next column:**
   - 3 + 4 = 7

5. **Next column:**
   - 6 + 5 = 11 (write down 1 and carry over 1)

6. **Next column:**
   - 5 + 3 + 1 (carry) = 9

7. **Next column:**
   - 1 + 8 = 9

8. **Next column:**
   - 6 + 5 = 11 (write down 1 and carry over 1)

9. **Next column:**
   - 3 + 2 + 1 (carry) = 6

10. **Next column:**
    - 9 + 7 = 16 (write down 6 and carry over 1)

11. **Next column:**
    - 4 + 4 